## Personal Dashboard
Goal of project:

1. Extract data from the API
2. Transform the data and make the format nice
3. Load the data once per day to a data warehouse or database so we can store some of it
4. visualize the data in a dashboard

data I am interested in
1. Weather - what is the weather in my location?
2. Give me a random recipe for dinner
3. any calendar events today? 
4. any birthdays?

In [1]:
#bring in necessary packages
import requests
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName('personal_dash').getOrCreate() #initializes a spark session
import json
#test again

In [2]:
#set the variables for our API call to the weather API
location = input("What city do you want to check the weather of? ")
query = {"query":location}
api_key = 'ccb8ee86c83d12b1216667cad1c3bdb1'
url = 'https://api.weatherstack.com/current?access_key='+api_key

In [3]:
# call the weather api - only 100 calls a month allowed
response = requests.get(url, params=query)
print(response.json())

{'request': {'type': 'City', 'query': 'Charlotte, United States of America', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Charlotte', 'country': 'United States of America', 'region': 'North Carolina', 'lat': '35.227', 'lon': '-80.843', 'timezone_id': 'America/New_York', 'localtime': '2024-12-14 13:37', 'localtime_epoch': 1734183420, 'utc_offset': '-5.0'}, 'current': {'observation_time': '06:37 PM', 'temperature': 9, 'weather_code': 116, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'], 'weather_descriptions': ['Partly cloudy'], 'wind_speed': 15, 'wind_degree': 54, 'wind_dir': 'NE', 'pressure': 1039, 'precip': 0, 'humidity': 50, 'cloudcover': 75, 'feelslike': 7, 'uv_index': 2, 'visibility': 16, 'is_day': 'yes'}}


In [4]:
#read results to a python dataframe
df_python = pd.json_normalize(response.json())
df_python.head()


,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,location.timezone_id,...,current.wind_degree,current.wind_dir,current.pressure,current.precip,current.humidity,current.cloudcover,current.feelslike,current.uv_index,current.visibility,current.is_day
0,City,"Charlotte, United States of America",en,m,Charlotte,United States of America,North Carolina,35.227,-80.843,America/New_York,...,54,NE,1039,0,50,75,7,2,16,yes


In [5]:
with open('api_output.json','w') as file:
    json.dump(response.json(),file)

In [6]:
#read results to a spark dataframe
df_spark = spark.read.json('api_output.json')
df_spark.show()

+--------------------+--------------------+--------------------+
|             current|            location|             request|
+--------------------+--------------------+--------------------+
|{75, 7, 50, yes, ...|{United States of...|{en, Charlotte, U...|
+--------------------+--------------------+--------------------+



In [8]:
response.json()

{'request': {'type': 'City',
  'query': 'Charlotte, United States of America',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Charlotte',
  'country': 'United States of America',
  'region': 'North Carolina',
  'lat': '35.227',
  'lon': '-80.843',
  'timezone_id': 'America/New_York',
  'localtime': '2024-12-14 13:37',
  'localtime_epoch': 1734183420,
  'utc_offset': '-5.0'},
 'current': {'observation_time': '06:37 PM',
  'temperature': 9,
  'weather_code': 116,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0002_sunny_intervals.png'],
  'weather_descriptions': ['Partly cloudy'],
  'wind_speed': 15,
  'wind_degree': 54,
  'wind_dir': 'NE',
  'pressure': 1039,
  'precip': 0,
  'humidity': 50,
  'cloudcover': 75,
  'feelslike': 7,
  'uv_index': 2,
  'visibility': 16,
  'is_day': 'yes'}}

In [ ]:
#read results to a spark dataframe
df_spark = spark.read.json('api_output.json')
subset = df_spark.select('location.*','current.*')
dim_location = df_spark.select('location.*')
fact_weather_daily = df_spark.select('current.*')


'''
#df_spark.show()
spark.sql("Select name as city \
                , region as state \
                , country \
                , timezone_id as timezone \
                from {subset}"
                ,subset=subset).show()
'''




+-------+--------+--------------------+---------------+
|   city|   state|             country|       timezone|
+-------+--------+--------------------+---------------+
|Chicago|Illinois|United States of ...|America/Chicago|
+-------+--------+--------------------+---------------+

